In [ ]:
# --- SETUP & IMPORTS ---
import timesfm  # Ensure you run: pip install git+https://github.com/google-research/timesfm.git
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Verify GPU
device = "gpu" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device.upper()}")

# --- CELL 2: LOAD TIMESFM 2.0 MODEL ---
# Official 2.0 Checkpoint (500M Parameters)
CHECKPOINT_REPO = "google/timesfm-2.0-500m-pytorch"

# Configuration for TimesFM 2.0
# Note: 2.0 supports up to 2048 context length (vs 512 in 1.0)
CONTEXT_LEN = 2048
HORIZON_LEN = 14       # Forecast horizon

print(f"Loading TimesFM 2.0 from {CHECKPOINT_REPO}...")

tfm = timesfm.TimesFm(
    context_len=CONTEXT_LEN,
    horizon_len=HORIZON_LEN,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=50,
    model_dims=1280,
    quantiles=[0.1, 0.5, 0.9],
    backend=device
)

# Load the 2.0 Weights
tfm.load_from_checkpoint(repo_id=CHECKPOINT_REPO)
print("✅ TimesFM 2.0 Model loaded successfully.")

In [ ]:
# --- DATA PREP ---
DATA_PATH = "../data/m4_processed.csv"  # Output from your data_prep.ipynb

try:
    df = pd.read_csv(DATA_PATH)
    print(f"✅ Loaded Data: {df.shape}")

    # Example Inference Call
    # Note: Ensure your DF has 'unique_id', 'ds', 'y' columns standard for TimesFM
    forecast_df = tfm.forecast_on_df(
        inputs=df,
        freq="D",  # D = Daily
        value_name="sales",
        num_jobs=-1
    )
    print("✅ Inference Complete")

except FileNotFoundError:
    print(f"❌ Error: Could not find {DATA_PATH}. Run 00_Data_Processing.ipynb first.")

In [ ]:
# --- EVALUATION & RESULTS ---

def calculate_metrics(y_true, y_pred, y_train_mean=1.0):
    """
    Standard Forecasting Metrics
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    # Avoid division by zero
    epsilon = 1e-10

    # MAE & MSE
    mae = np.mean(np.abs(y_true - y_pred))
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)

    # sMAPE (Symmetric Mean Absolute Percentage Error)
    smape = 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + epsilon))

    # MASE (Mean Absolute Scaled Error)
    # Note: Strictly speaking, MASE requires the Naive error from the training set.
    # For Zero-Shot benchmarks, we often approximate the scale using the mean value
    # or the simple MAE if training data isn't available in memory.
    # Here we use the mean of the actuals to normalize, which is a common proxy for magnitude.
    mase = mae / (np.mean(np.abs(y_true)) + epsilon)

    return {"MAE": mae, "MSE": mse, "RMSE": rmse, "sMAPE": smape, "MASE": mase}

print("\n--- Processing Results ---")

# 1. Standardize Columns for Merge
# Ensure both have 'unique_id' and 'ds' (date) columns
if 'series_id' in df.columns:
    df = df.rename(columns={'series_id': 'unique_id'})
if 'date' in df.columns:
    df = df.rename(columns={'date': 'ds'})

# 2. Merge Forecast with Ground Truth
# TimesFM output usually has columns: ['unique_id', 'ds', 'timesfm']
results_df = df.merge(forecast_df, on=['unique_id', 'ds'], how='inner')

# 3. Calculate Metrics
# Assuming your ground truth column is named 'sales' or 'value'.
# We try both just in case.
target_col = 'sales' if 'sales' in results_df.columns else 'value'
pred_col = 'timesfm' # Default output column from TimesFM

if target_col in results_df.columns and pred_col in results_df.columns:
    metrics = calculate_metrics(results_df[target_col], results_df[pred_col])

    print("\n" + "="*40)
    print(f"🏆 FINAL RESULTS ({target_col.upper()})")
    print("="*40)
    print(f"MAE:   {metrics['MAE']:.4f}")
    print(f"MSE:   {metrics['MSE']:.4f}")
    print(f"RMSE:  {metrics['RMSE']:.4f}")
    print(f"sMAPE: {metrics['sMAPE']:.4f}%")
    print(f"MASE:  {metrics['MASE']:.4f} (Approx)")
    print("="*40)

    # Optional: Plot the first series to visually verify
    print("\nVisualizing first series...")
    first_id = results_df['unique_id'].unique()[0]
    sample = results_df[results_df['unique_id'] == first_id].tail(100)

    plt.figure(figsize=(12, 6))
    plt.plot(sample['ds'], sample[target_col], label='Actual', color='black')
    plt.plot(sample['ds'], sample[pred_col], label='TimesFM Forecast', color='blue', linestyle='--')
    plt.title(f"Forecast vs Actual: {first_id}")
    plt.legend()
    plt.show()

else:
    print(f"❌ Error: Could not find target column '{target_col}' or prediction column '{pred_col}' in merged data.")
    print("Columns found:", results_df.columns)